In [ ]:
# Vizro-AI usage
import pandas as pd

df = pd.read_csv('website_traffic_data.csv')
df["Date"] = pd.to_datetime(df['Date'],dayfirst=True)

In [ ]:
import vizro_ai
from vizro_ai import VizroAI
from dotenv import load_dotenv

print(vizro_ai.__version__)
load_dotenv()

vizro_ai = VizroAI(model="gpt-4-0613")

In [ ]:
# Compare latest and stable by date
compare_versions_fig = vizro_ai.plot(df, "Combine rows of Views for each Date for latest and stable Version. Draw a smoothed line graph comparing Views per Date for latest and stable.", explain=True)
compare_versions_fig.show()

In [ ]:
# Total views of the top performing stable pages
top_performing_pages_fig = vizro_ai.plot(df, "Collate rows of data for Path where Version==stable. Create a horizontal bar chart describing the total Views for the top 5 pages. Add the numbers to each bar. Decrease font size of marks", explain=True)
top_performing_pages_fig.show()

In [ ]:
# Top 5 performing pages on stable by date
top_pages_by_date_fig = vizro_ai.plot(df, "Collate rows of data for Path where Version==stable. Create a line graph for the total Views per Date for the top 5 Paths", explain=True)
top_pages_by_date_fig.show()

In [ ]:
# Dashboard using Vizro-AI calls only
from vizro import Vizro
import vizro.models as vm
import vizro.plotly.express as px

first_page = vm.Page(
title="Documentation traffic dashboard",
layout=vm.Layout(
    grid=[
        [0, 0, 2, 2],
        [0, 0, 2, 2], 
        [0, 0, 2, 2], 
        [1, 1, 2, 2],
        [1, 1, 2, 2],
        [1, 1, 2, 2],
    ],
),
components=[
    # Latest and stable by date
    vm.Graph(id='version_views_graph', figure=compare_versions_fig),
    
    # Top performing stable pages
    vm.Graph(id='popular_pages_chart_stable', figure=top_performing_pages_fig),

    # Top 5 stable pages by date
    vm.Graph(id='popular_pages_graph_stable', figure=top_pages_by_date_fig),
    ],
)
dashboard = vm.Dashboard(pages=[first_page])
Vizro().build(dashboard).run(port=8006)

In [ ]:
# Dashboard from Vizro-AI calls and using Vizro-AI generated chart code
from vizro import Vizro
import vizro.models as vm
import vizro.plotly.express as px
from vizro.models.types import capture
import plotly.graph_objects as go
import pandas as pd

@capture('graph')
def compare_versions_custom_chart(data_frame=None):
    if data_frame is None:
        data_frame = pd.DataFrame()
    
    data_frame['Date'] = pd.to_datetime(data_frame['Date'])
    # Group by Date and Version and sum the Views
    aggregated_df = data_frame.groupby(['Date', 'Version'])['Views'].sum().reset_index()
    # Pivot the dataframe to have Versions as columns
    pivot_df = aggregated_df.pivot(index='Date', columns='Version', values='Views')
    # Fill NaN values with 0
    pivot_df.fillna(0, inplace=True)
    data_frame = pivot_df.reset_index()

    # Create a line trace for each version
    trace1 = go.Scatter(x=data_frame['Date'], y=data_frame['latest'], mode='lines', name='latest', line=dict(color='#689F38'))
    trace2 = go.Scatter(x=data_frame['Date'], y=data_frame['stable'], mode='lines', name='stable', line=dict(color='#FDC935'))

    # Define the layout
    layout = go.Layout(title='Views per Date for latest and stable Version', xaxis=dict(title='Date'), yaxis=dict(title='Views'))

    # Create a Figure and add the traces
    fig = go.Figure(data=[trace1, trace2], layout=layout)

    # Return the figure
    return fig   


compare_versions_custom_fig = compare_versions_custom_chart(data_frame=df)

first_page = vm.Page(
title="Documentation traffic dashboard",
layout=vm.Layout(
    grid=[
        [0, 0, 2, 2],
        [0, 0, 2, 2], 
        [0, 0, 2, 2], 
        [1, 1, 2, 2],
        [1, 1, 2, 2],
        [1, 1, 2, 2],
    ],
),
components=[
    # Latest and stable by date
    vm.Graph(id='version_views_graph', figure=compare_versions_custom_fig),
    
    # Top performing stable pages
    vm.Graph(id='popular_pages_chart_stable', figure=top_performing_pages_fig),

    # Top 5 stable pages by date
    vm.Graph(id='popular_pages_graph_stable', figure=top_pages_by_date_fig),
    ],
)
dashboard = vm.Dashboard(pages=[first_page])
Vizro().build(dashboard).run(port=8006)